# Bufstream Iceberg quickstart

In [ ]:
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('Jupyter')
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .set("spark.sql.catalog.bufstream-quickstart", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.bufstream-quickstart.type", "rest")
        .set("spark.sql.catalog.bufstream-quickstart.uri", "http://iceberg-rest:8181")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [1]:
%%sql
WITH categories AS (
    SELECT 
        line_item.product.category.name AS category,
        SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS revenue,
        COUNT(DISTINCT val.cart_id) AS carts,
        SUM(line_item.quantity) AS units_sold
    FROM `bufstream`.`orders`
    LATERAL VIEW EXPLODE(val.line_items) AS line_item
    GROUP BY category
),
totals AS (
    SELECT 
        'TOTAL' AS category,
        SUM(revenue) AS revenue,
        -- We can't just SUM() carts: one cart may have items from multiple categories.
        (SELECT(COUNT(DISTINCT val.cart_id)) AS carts FROM `bufstream`.`orders`) AS carts,
        SUM(units_sold) AS units_sold
    FROM categories
)
SELECT
    category,
    FORMAT_NUMBER(revenue, 2) as revenue,
    FORMAT_NUMBER(carts, 0) as carts,
    FORMAT_NUMBER(units_sold, 0) as units_sold
FROM (
    SELECT * FROM categories
    UNION ALL
    SELECT * FROM totals
)    
ORDER BY 
    CASE WHEN category = 'TOTAL' THEN 1 ELSE 0 END,
    category ASC;


25/11/05 13:03:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

category,revenue,carts,units_sold
Books & Stationery,"932,889.31","7,958","34,769"
Electronics & Accessories,"1,219,922.47","8,773","41,753"
Home & Garden,"1,369,609.23","8,759","42,177"
Kitchen & Dining,"1,307,312.22","7,966","34,878"
Personal Care,"763,054.77","7,107","28,423"
Sports & Outdoors,"1,630,854.55","8,840","41,645"
TOTAL,"7,223,642.55","13,647","223,645"
